In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sns.set_style('whitegrid')

In [2]:
file1 = "../data/raw/loan_sample.csv"
ChunkSize = 200000
i = 1
for chunk in pd.read_csv(file1, chunksize=ChunkSize, low_memory=False):
    loan_data = chunk if i == 1 else pd.concat([loan_data, chunk])
    print('-->Read Chunk...', i)
    i += 1

-->Read Chunk... 1
-->Read Chunk... 2
-->Read Chunk... 3
-->Read Chunk... 4
-->Read Chunk... 5


In [8]:
# print((data.isna().sum()[data.isna().sum() > 0]))
df_null = pd.DataFrame({'Count': loan_data.isnull().sum(), 'Percent': 100*loan_data.isnull().sum()/len(loan_data)})
print(df_null[df_null['Percent'] >= 10].sort_values(by='Percent', ascending=False).round(1))

                         Count  Percent
next_pymnt_d            521326     57.7
mths_since_last_delinq  463489     51.3
il_util                 427560     47.3
mths_since_rcnt_il      363938     40.2
all_util                346556     38.3
open_act_il             346468     38.3
open_il_12m             346468     38.3
open_il_24m             346468     38.3
total_bal_il            346468     38.3
open_acc_6m             346468     38.3
open_rv_12m             346468     38.3
open_rv_24m             346468     38.3
max_bal_bc              346468     38.3
inq_fi                  346468     38.3
total_cu_tl             346468     38.3
inq_last_12m            346468     38.3
mths_since_recent_inq   118503     13.1


## Step 5A : Checking data in columns with missing data more than 10% 
Lets examine few values of this columns so we can get idea what kind of data estimation we need to do.

In [10]:
col_10 = list(df_null[df_null['Percent'] > 10].index)
print(loan_data[col_10].head(3))

   mths_since_last_delinq next_pymnt_d  open_acc_6m  open_act_il  open_il_12m  \
0                     NaN          NaN          0.0          0.0          0.0   
1                    50.0          NaN          0.0          1.0          0.0   
2                     NaN          NaN          2.0          2.0          0.0   

   open_il_24m  mths_since_rcnt_il  total_bal_il  il_util  open_rv_12m  \
0          0.0                 NaN           0.0      NaN          0.0   
1          0.0                45.0       57500.0      NaN          0.0   
2          0.0                28.0        9283.0     32.0          2.0   

   open_rv_24m  max_bal_bc  all_util  inq_fi  total_cu_tl  inq_last_12m  \
0          0.0      4600.0      37.0     0.0          0.0           0.0   
1          2.0      3739.0      32.0     0.0          0.0           0.0   
2          5.0      5395.0      57.0     0.0          0.0           4.0   

   mths_since_recent_inq  
0                    NaN  
1                    Na

## Step 5B : Dropping payment dates columns 
As we see above data set most missing data is numeric and we can use mode or median in order to fill missing values. By looking at dataset and name of columns, mode will be better option compared to median for most of columns. For "total_bil_il" we will use median. Lets analyze mode and median of those columns. But before we need to drop 3 columns which is not useful for our analysis. Those are 1) "next_pymnt_dt 2) 'last_credit_pull_d' 3) 'last_pymnt_d'
Lets create a list of these columns.

In [12]:
# print(df_clean.shape)
cols_to_drop = ['next_pymnt_d', 'last_credit_pull_d', 'last_pymnt_d']
loan_data.drop(cols_to_drop, axis=1, inplace=True)
df_null = pd.DataFrame({'Count': loan_data.isnull().sum(), 'Percent': 100*loan_data.isnull().sum()/len(loan_data)})

## Step 5C : Checking mode and median of missing data columns 
Now we will eamine mode and median of all columns those we trying to do estiamtion.

In [15]:
col_10 = list(df_null[df_null['Percent'] > 10].index)
# print(col_10)
# print(df_null.loc[col_10, :])
for index in col_10:
    df_null.loc[index, 'mode'] = loan_data[index].mode()[0]
    df_null.loc[index, 'median'] = loan_data[index].median()
    df_null.loc[index, 'mean'] = loan_data[index].mean()
print(df_null.loc[col_10, :].round(1))


                         Count  Percent  mode   median     mean
mths_since_last_delinq  463489     51.3  12.0     31.0     34.6
open_acc_6m             346468     38.3   0.0      1.0      0.9
open_act_il             346468     38.3   1.0      2.0      2.8
open_il_12m             346468     38.3   0.0      0.0      0.7
open_il_24m             346468     38.3   1.0      1.0      1.6
mths_since_rcnt_il      363938     40.2   7.0     13.0     21.2
total_bal_il            346468     38.3   0.0  23108.0  35503.3
il_util                 427560     47.3  78.0     72.0     69.1
open_rv_12m             346468     38.3   0.0      1.0      1.3
open_rv_24m             346468     38.3   1.0      2.0      2.8
max_bal_bc              346468     38.3   0.0   4422.0   5813.6
all_util                346556     38.3  59.0     58.0     57.0
inq_fi                  346468     38.3   0.0      1.0      1.0
total_cu_tl             346468     38.3   0.0      0.0      1.5
inq_last_12m            346468     38.3 

## Step 5D : predicting missing values using mode 
By looking at above table, it makes more sense to fill all above columns by mode rather than median. Lets fill those missing values in above columns using mode.

In [16]:
loan_data[col_10] = loan_data[col_10].fillna(loan_data.mode().iloc[0])

## Step 6 : Dropping all missing data 
Now lets examine remaining missing data. Most columns will have missing data less than 10%. We will simply drop them. We may see lot of columns having missing data less than 10%. We will simply drop them. That way we will only lose 10% of data

In [20]:
df_clean = loan_data.dropna()
print(loan_data.shape)


(741542, 100)


## Get year info from Issue Date

In [21]:
df_clean['year'] = pd.to_datetime(df_clean['issue_d']).dt.year
df_clean = df_clean.drop('issue_d', axis=1)
df_clean['year'].head()

1    2016
2    2016
3    2017
4    2017
5    2013
Name: year, dtype: int64

In [22]:
df_clean['percent_complete'] = (df_clean['total_pymnt']*100 / df_clean['loan_amnt']).round(2)
# print(df_clean[df_clean['loan_status'] == "Fully Paid"]['percent_complete'].head())

In [23]:
df_clean['open_acc_6m'] = df_clean['open_acc_6m'].apply(lambda x: 3 if x>=3 else x)
df_clean['open_act_il'] = df_clean['open_act_il'].apply(lambda x: 5 if x>=5 else x)
df_clean['open_il_12m'] = df_clean['open_il_12m'].apply(lambda x: 3 if x>=3 else x)
df_clean['open_rv_12m'] = df_clean['open_rv_12m'].apply(lambda x: 3 if x>=3 else x)
df_clean['inq_fi'] = df_clean['inq_fi'].apply(lambda x: 3 if x>=3 else x)
df_clean['inq_last_12m'] = df_clean['inq_last_12m'].apply(lambda x: 4 if x>=4 else x)
df_clean['acc_open_past_24mths'] = df_clean['acc_open_past_24mths'].apply(lambda x: 8 if x>=8 else x)
df_clean['mort_acc'] = df_clean['mort_acc'].apply(lambda x: 4 if x>=4 else x)
df_clean['num_accts_ever_120_pd'] = df_clean['num_accts_ever_120_pd'].apply(lambda x: 3 if x>=3 else x)
df_clean['num_tl_op_past_12m'] = df_clean['num_tl_op_past_12m'].apply(lambda x: 5 if x>=5 else x)

In [24]:
df_clean['max_bal_bc_cat'] = pd.cut(x=df_clean['max_bal_bc'], bins=[0, 1000, 2000, 5000, 10000 ,1000000],\
                                    include_lowest=True)
df_clean['num_sats_cat'] = pd.cut(x=df_clean['num_sats'], bins=[0, 5, 10, 20, 30 ,150],include_lowest=True)
df_clean['pct_tl_nvr_dlq_cat'] = pd.cut(x=df_clean['pct_tl_nvr_dlq'], bins=[0, 60, 70, 80, 90 ,100],include_lowest=True)
df_clean['percent_bc_gt_75_cat'] = pd.cut(x=df_clean['percent_bc_gt_75'], bins=[0, 20, 40, 60, 80 ,100],include_lowest=True)
df_clean['percent_bc_gt_75_cat'].head()

1      (20.0, 40.0]
2     (80.0, 100.0]
3      (20.0, 40.0]
4    (-0.001, 20.0]
5    (-0.001, 20.0]
Name: percent_bc_gt_75_cat, dtype: category
Categories (5, interval[float64]): [(-0.001, 20.0] < (20.0, 40.0] < (40.0, 60.0] < (60.0, 80.0] < (80.0, 100.0]]

## Finding Out redundant values
We will use value_count(normalize=True) function in order to find out redundant values

In [25]:
col_list = ['mo_sin_old_il_acct', 'mort_acc', 'num_accts_ever_120_pd', 'num_rev_accts', 'num_rev_tl_bal_gt_0',\
            'num_sats_cat', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', \
           'pct_tl_nvr_dlq_cat', 'percent_bc_gt_75_cat', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', \
           'hardship_flag']
# print(col_list)
print(df_clean['hardship_flag'].value_counts(normalize=True).round(4)*100)
# print(df_clean['num_sats'].nlargest(5))
for c in col_list:
    print ("---- %s ---" % c)
    df_t = df_clean[c].value_counts(normalize=True).round(2)
    
    print("Max value = %f" %df_t.max())
    print("Length = %d" %len(df_t))

N    99.96
Y     0.04
Name: hardship_flag, dtype: float64
---- mo_sin_old_il_acct ---
Max value = 0.010000
Length = 495
---- mort_acc ---
Max value = 0.410000
Length = 5
---- num_accts_ever_120_pd ---
Max value = 0.760000
Length = 4
---- num_rev_accts ---
Max value = 0.060000
Length = 105
---- num_rev_tl_bal_gt_0 ---
Max value = 0.160000
Length = 45
---- num_sats_cat ---
Max value = 0.450000
Length = 5
---- num_tl_120dpd_2m ---
Max value = 1.000000
Length = 5
---- num_tl_30dpd ---
Max value = 1.000000
Length = 5
---- num_tl_90g_dpd_24m ---
Max value = 0.950000
Length = 26
---- num_tl_op_past_12m ---
Max value = 0.250000
Length = 6
---- pct_tl_nvr_dlq_cat ---
Max value = 0.780000
Length = 5
---- percent_bc_gt_75_cat ---
Max value = 0.360000
Length = 5
---- pub_rec_bankruptcies ---
Max value = 0.880000
Length = 12
---- tax_liens ---
Max value = 0.970000
Length = 34
---- tot_hi_cred_lim ---
Max value = 0.000000
Length = 341890
---- hardship_flag ---
Max value = 1.000000
Length = 2


In [26]:
col_list2 = ['pymnt_plan', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',\
            'total_acc', 'out_prncp', 'out_prncp_inv',  'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', \
             'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt',\
            'collections_12_mths_ex_med', 'policy_code',  'application_type',  'acc_now_delinq', 'tot_coll_amt', \
            'tot_cur_bal', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'open_rv_24m', 'max_bal_bc', \
             'all_util', 'total_rev_hi_lim', 'total_cu_tl', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', \
             'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',\
             'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_bc_tl',\
            'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', \
            'num_rev_tl_bal_gt_0', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'pct_tl_nvr_dlq_cat',\
            'pct_tl_nvr_dlq', 'pub_rec_bankruptcies', 'tax_liens', 'total_il_high_credit_limit', 'hardship_flag',\
            'disbursement_method', 'debt_settlement_flag', 'max_bal_bc', 'num_sats', 'pct_tl_nvr_dlq', 'percent_bc_gt_75',\
            'funded_amnt_inv', 'zip_code', 'funded_amnt',  'title', 'earliest_cr_line', ]
print(len(col_list2))
# df_clean['pymnt_plan'].value_counts(normalize=True)

71


## Step 7 : Analyzing datatypes of clean data 
Now we have get rid of all missing data. Dataset is almost clean. Lets see datatypes of datasets.

In [27]:
# print(df_clean.dtypes)

df_clean2 = df_clean.drop(col_list2, axis=1) 

print(df_clean2.dtypes)

loan_amnt                   int64
term                       object
int_rate                  float64
installment               float64
grade                      object
sub_grade                  object
emp_title                  object
emp_length                 object
home_ownership             object
annual_inc                float64
verification_status        object
loan_status                object
purpose                    object
addr_state                 object
dti                       float64
delinq_2yrs               float64
initial_list_status        object
open_acc_6m               float64
open_act_il               float64
open_il_12m               float64
il_util                   float64
open_rv_12m               float64
inq_fi                    float64
inq_last_12m              float64
acc_open_past_24mths      float64
mort_acc                  float64
num_accts_ever_120_pd     float64
num_tl_op_past_12m        float64
tot_hi_cred_lim           float64
total_bal_ex_m

## Step 8 : Fixing "Employment Length" column 
In above datatypes one of the important title is employment length. We should be expecting it as integer or float but it is object type. Lets analyze this column.

In [28]:
print(df_clean2['emp_length'].value_counts())

10+ years    266035
2 years       71507
3 years       63512
< 1 year      63033
1 year        51543
5 years       48532
4 years       47712
6 years       35884
7 years       32829
8 years       32665
9 years       28290
Name: emp_length, dtype: int64


In [29]:
# df_clean['emp_length'].astype('str').dtypes
df_clean2['emp_length'] = df_clean2['emp_length'].str.replace(r'[+]\s|[a-z]', '')
df_clean2['emp_length'] = df_clean2['emp_length'].str.replace('< 1', '0')
# df_clean['emp_length'] = df_clean['emp_length'].str.replace('10+ ', '10')

print(df_clean2['emp_length'].value_counts())

10    266035
2      71507
3      63512
0      63033
1      51543
5      48532
4      47712
6      35884
7      32829
8      32665
9      28290
Name: emp_length, dtype: int64


In [30]:
# df_clean2['year'] = pd.to_datetime(df_clean2['issue_d']).dt.year
# print(train_data['loan_status'].value_counts())
# train_data['status_var'] = train_data['loan_status'].map({'Fully Paid':0, 'Charged Off':1, 'Late (31-120 days)':1, \
#                                                         'In Grace Period':1, 'Late (16-30 days)':1, 'Default':1})
# print(train_data['status_var'].value_counts())
loan_status_dict = {"Fully Paid": 0, "Charged Off": 1, "Late (31-120 days)": 1,
                    "In Grace Period": 1, "Late (16-30 days)": 1, "Default": 1, "Current" : 2}
# known_status = df_clean2['loan_status'] != 'Current'
df_clean2["loan_status_count"] = df_clean2["loan_status"].map(loan_status_dict)
df_clean2["loan_status_count"] = df_clean2["loan_status_count"].astype("int")

# loan_known_data = df_clean2[known_status].reset_index()
loan_status_dict = {0.0: "Good loans", 1.0: "Bad loans", 2.0: 'Unknown'}
df_clean2["loan_status"] = df_clean2["loan_status_count"].map(loan_status_dict)

In [31]:
# df_clean3 = df_clean2.drop('loan_status')
print(df_clean2.shape)
pd.options.display.float_format = '{:.2f}'.format
print(df_clean2.head())

(741542, 37)
   loan_amnt        term  int_rate  installment grade sub_grade  \
1       8400   36 months     10.75       274.02     B        B4   
2       6000   36 months     17.27       214.73     D        D2   
3      15000   36 months      9.93       483.52     B        B2   
4       5600   36 months      7.35       173.81     A        A4   
5      12000   36 months     11.55       396.00     B        B3   

                  emp_title emp_length home_ownership  annual_inc  \
1  Administrative Assistant         4            RENT    49999.00   
2                   Teacher         10            OWN    72000.00   
3                 Professor         1        MORTGAGE    80000.00   
4        Software Developer         4            RENT    78618.00   
5                      ucsf         1            RENT   120000.00   

  verification_status loan_status             purpose addr_state   dti  \
1            Verified   Bad loans  debt_consolidation         NY  4.61   
2        Not Verified

In [32]:
df_clean2.to_csv("../data/processed/Loan_cleandata2.csv", index=False)